In [2]:
import tensorflow as tf
import pandas as pd
import numpy

In [3]:
data = pd.read_csv("Eng-Tel.csv")

In [4]:
data.head()

,English Word,Telugu Translation
0,The,ది
1,Lion,సింహం
2,King,రాజు
3,is,ఉంది
4,American,అమెరికన్


Tokenizer for languages

In [5]:
Tokenizer_Eng = tf.keras.preprocessing.text.Tokenizer()
Tokenizer_Tel = tf.keras.preprocessing.text.Tokenizer()

In [6]:
Tokenizer_Eng.fit_on_texts(data['English Word'])
Tokenizer_Tel.fit_on_texts(data['Telugu Translation'])

Converting text to Sequences

In [7]:
train_Eng = Tokenizer_Eng.texts_to_sequences(data['English Word'])
train_Tel = Tokenizer_Tel.texts_to_sequences(data['Telugu Translation'])

Padding or Pad Sequences

In [8]:
trainEng = tf.keras.preprocessing.sequence.pad_sequences(train_Eng , padding='post')
trainTel = tf.keras.preprocessing.sequence.pad_sequences(train_Tel , padding='post')

Model Declaration

In [11]:
Eng_vocab_size = len(Tokenizer_Eng.word_index)+1
Tel_vocab_size = len(Tokenizer_Tel.word_index)+1
embedding_dim = 64
units = 128

Encoder Model

In [12]:
encoder_inputs = tf.keras.Input(shape=(None,))
enc_emb = tf.keras.layers.Embedding(Eng_vocab_size,embedding_dim)(encoder_inputs)
enc_output , state_h , state_c = tf.keras.layers.LSTM(units, return_state=True)(enc_emb)

Decoder Model

In [14]:
decoder_inputs = tf.keras.Input(shape=(None,))
dec_emb = tf.keras.layers.Embedding(Tel_vocab_size, embedding_dim)(decoder_inputs)
dec_lstm = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True)
dec_output, _, _ = dec_lstm(dec_emb, initial_state=[state_h, state_c])
decoder_dense = tf.keras.layers.Dense(Tel_vocab_size, activation='softmax')
output = decoder_dense(dec_output)

In [15]:
model = tf.keras.Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.fit([trainEng, trainTel[:, :-1]], trainTel[:, 1:], 
          batch_size=64,validation_split=0.2, epochs=20)

Epoch 1/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.8593 - loss: 7.5303 - val_accuracy: 0.9814 - val_loss: 3.5404
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9820 - loss: 2.6791 - val_accuracy: 0.9814 - val_loss: 0.7159
Epoch 3/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.9812 - loss: 0.1759 - val_accuracy: 0.9814 - val_loss: 0.4465
Epoch 4/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9763 - loss: 0.1639 - val_accuracy: 0.9814 - val_loss: 0.4523
Epoch 5/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.9836 - loss: 0.1055 - val_accuracy: 0.9814 - val_loss: 0.4345
Epoch 6/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.9844 - loss: 0.0971 - val_accuracy: 0.9814 - val_loss: 0.4163
Epoch 7/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.9792 - loss: 0.1280 - val_accuracy: 0.9814 - val_loss: 0.3946
Epoch 8/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9814 - loss: 0.1085 - val_accuracy: 0.9814 - v

In [26]:
model.save("model.keras")

In [45]:
data2 = pd.read_csv('hello.csv')

In [46]:
data2.head()

,words,sentences
0,"ది,లయన్,కింగ్,అనేది,2019,అమెరికన్,మ్యూజికల్,డ్...",ది లయన్ కింగ్ అనేది 2019 అమెరికన్ మ్యూజికల్ డ్...
1,"దీనికి,జెఫ్,నాథన్సన్,రాసిన,జోన్,ఫావ్‌రో,దర్శకత...",దీనికి జెఫ్ నాథన్సన్ రాసిన జోన్ ఫావ్‌రో దర్శకత...
2,"ఈ,చిత్రంలో,డొనాల్డ్,గ్లోవర్,సేథ్,రోజెన్,చివెటె...",ఈ చిత్రంలో డొనాల్డ్ గ్లోవర్ సేథ్ రోజెన్ చివెటె...
3,"ఈ,కథాంశం,సింబా,ఒక,యువ,సింహాన్ని,అనుసరిస్తుంది,...","ఈ కథాంశం సింబా ఒక యువ సింహాన్ని అనుసరిస్తుంది,..."
4,"1994,ల,రీమేక్,కోసం,ప్రణాళికలు,ది,జంగిల్,బుక్,2...",1994 ల రీమేక్ కోసం ప్రణాళికలు ది జంగిల్ బుక్ 2...


In [47]:
def split_and_shuffle(data):
  return random.sample(data.split(','), len(data.split(',')))

In [50]:
data2['words'] = data2['words'].apply(split_and_shuffle)

In [51]:
data2.head()

,words,sentences
0,"[ఫోటోరియలిస్టిక్, మ్యూజికల్, అనేది, అమెరికన్, ...",ది లయన్ కింగ్ అనేది 2019 అమెరికన్ మ్యూజికల్ డ్...
1,"[మరియు, రాసిన, జోన్, ఫావ్‌రో, డిస్నీ, నాథన్సన్...",దీనికి జెఫ్ నాథన్సన్ రాసిన జోన్ ఫావ్‌రో దర్శకత...
2,"[పాత్రను, రైట్, ఫ్లోరెన్స్, జోసెఫ్, తన, ఒలివర్...",ఈ చిత్రంలో డొనాల్డ్ గ్లోవర్ సేథ్ రోజెన్ చివెటె...
3,"[మాతృభూమికి, యువ, హత్య, చేతిలో, తన, సింహాన్ని,...","ఈ కథాంశం సింబా ఒక యువ సింహాన్ని అనుసరిస్తుంది,..."
4,"[2016, ది, జంగిల్, విజయాల, ప్రణాళికలు, డిస్నీ,...",1994 ల రీమేక్ కోసం ప్రణాళికలు ది జంగిల్ బుక్ 2...


In [52]:
tokenizer_Words = tf.keras.preprocessing.text.Tokenizer()
tokenizer_Sen = tf.keras.preprocessing.text.Tokenizer()

In [54]:
tokenizer_Words.fit_on_texts(data2['words'])
tokenizer_Sen.fit_on_texts(data2['sentences'])

In [56]:
train_input_seq = tokenizer_Words.texts_to_sequences(data2['words'])
train_output_seq = tokenizer_Sen.texts_to_sequences(data2['sentences'])

In [57]:
train_input_seq = tf.keras.preprocessing.sequence.pad_sequences(train_input_seq, padding='post')
train_output_seq = tf.keras.preprocessing.sequence.pad_sequences(train_output_seq, padding='post')

Model Creation

In [58]:
input_vocab_size = len(tokenizer_Words.word_index) + 1
output_vocab_size = len(tokenizer_Sen.word_index) + 1
embedding_dim = 64
units = 128

In [59]:
encoder_inputs = tf.keras.Input(shape=(None,))
enc_emb = tf.keras.layers.Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
enc_output, state_h, state_c = tf.keras.layers.LSTM(units, return_state=True)(enc_emb)

In [60]:
decoder_inputs = tf.keras.Input(shape=(None,))
dec_emb = tf.keras.layers.Embedding(output_vocab_size, embedding_dim)(decoder_inputs)
dec_lstm = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True)
dec_output, _, _ = dec_lstm(dec_emb, initial_state=[state_h, state_c])
decoder_dense = tf.keras.layers.Dense(output_vocab_size, activation='softmax')
output = decoder_dense(dec_output)

In [81]:
model2 = tf.keras.Model([encoder_inputs, decoder_inputs], output)
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [82]:
batch_size = 64
epochs = 20

In [83]:
model2.fit([train_input_seq, train_output_seq[:, :-1]], train_output_seq[:, 1:], 
          validation_split=0.2,
          batch_size=batch_size, epochs=epochs)

Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 374ms/step - accuracy: 0.7737 - loss: 1.7426 - val_accuracy: 0.7455 - val_loss: 2.1943
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 356ms/step - accuracy: 0.7726 - loss: 1.7307 - val_accuracy: 0.7454 - val_loss: 2.2207
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 338ms/step - accuracy: 0.7774 - loss: 1.6720 - val_accuracy: 0.7455 - val_loss: 2.2507
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 337ms/step - accuracy: 0.7670 - loss: 1.7428 - val_accuracy: 0.7447 - val_loss: 2.2987
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 336ms/step - accuracy: 0.7722 - loss: 1.6927 - val_accuracy: 0.7456 - val_loss: 2.3009
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 344ms/step - accuracy: 0.7807 - loss: 1.6193 - val_accuracy: 0.7456 - val_loss: 2.3285
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 327ms/step - accuracy: 0.7797 - loss: 1.6095 - val_accuracy: 0.7452 - val_loss: 2.3559
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 337ms/step - accuracy: 0.7793 - loss: 1.6013 - val_accuracy: 0.